# TensorFlow

*TensorFlow* is a powerful open source software library for numerical computation, particularly well suited and fine-tuned for large-scale Machine Learning. Its basic principle is simple: you first define in Python a graph of computations to perform, and then TensorFlow takes that graph and runs it efficiently using optimized C++ code. Most importantly, it is possible to break up the graph into several chunks and run them in parallel across multiple CPUs or GPUs.

In [3]:
import tensorflow as tf

C:\Users\hheim\Anaconda3\envs\TensorFlow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
!conda list

# packages in environment at C:\Users\hheim\Anaconda3\envs\TensorFlow:
#
# Name                    Version                   Build  Channel
_tflow_select             2.1.0                       gpu    anaconda
absl-py                   0.9.0                    py36_0  
argon2-cffi               20.1.0                   pypi_0    pypi
astor                     0.8.1                    py36_0  
attrs                     19.3.0                   pypi_0    pypi
backcall                  0.2.0                      py_0  
blas                      1.0                         mkl  
bleach                    3.1.5                    pypi_0    pypi
ca-certificates           2020.6.24                     0  
certifi                   2020.6.20                py36_0  
cffi                      1.14.2                   pypi_0    pypi
colorama                  0.4.3                      py_0  
cudatoolkit               9.0                           1    anaconda
decorator                 4.4.2     

h5py                      2.7.1            py36he54a1c3_0    anaconda
hdf5                      1.10.1           vc14hb361328_0  [vc14]  anaconda
icc_rt                    2019.0.0             h0cc432a_1    anaconda
importlib-metadata        1.7.0                    py36_0  
intel-openmp              2020.1                      216  
ipykernel                 5.3.4            py36h5ca1d4c_0  
ipython                   7.16.1           py36h5ca1d4c_0  
ipython_genutils          0.2.0                    py36_0  
ipywidgets                7.5.1                    pypi_0    pypi
jedi                      0.17.2                   py36_0  
jinja2                    2.11.2                   pypi_0    pypi
jsonschema                3.2.0                    pypi_0    pypi
jupyter                   1.0.0                    pypi_0    pypi
jupyter-console           6.1.0                    pypi_0    pypi
jupyter_client            6.1.6                      py_0  
jupyter_core              4.6.3   

In [7]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

The most important thing to understand is that this code does not actually perform any computation, even though it looks like it does (especially the last line). It just creates a computation graph. In fact, even the variables are not initialized yet. To evaluate this graph, you need to open a TensorFlow session and use it to initialize the variables and evaluate.

A variable starts its life when its initializer is run, and it ends when the session is closed.

In [13]:
with tf.Session() as sess:
    sess.run(x.initializer)
    sess.run(y.initializer)
    res = sess.run(f)
    print(res)
    
tf.reset_default_graph()

42


Any instance of a node that is created is added to the default graph. If you want separate graphs do the following:

**TIP**

In Jupyter (or in a Python shell), it is common to run the same commands more than once while you are experimenting. As a
result, you may end up with a default graph containing many duplicate nodes. One solution is to restart the Jupyter kernel (or the Python shell), but a more convenient solution is to just reset the default graph by running tf.reset_default_graph().

In [37]:
graph_1 = tf.Graph()

with graph_1.as_default():
    x1 = tf.Variable(1)

## Linear Regression with TensorFlow

In the Python API tensors are simply represented by NumPy ndarrays, you can also use `transpose()`, `matmul()`, and `matrix_inverse()`.

In [83]:
import numpy as np

np.random.seed(1234)

X=np.random.rand(1000,3)
B=np.array([1,2,3]).reshape(3,1)
Y=X@B+np.random.randn(1000,1)*5
B_hat = np.linalg.inv(X.transpose()@X)@X.transpose()@Y


Xt = tf.constant(X, dtype=tf.float32, name='X') #Source operation
Yt = tf.constant(Y, dtype=tf.float32, name='Y') #Source operation
Xt_T = tf.transpose(Xt)
Betas = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(Xt_T,Xt)),Xt_T),Yt)

with tf.Session() as LR_sess:
    Betas_LR = Betas.eval()
    Betas_LR = Betas_LR.reshape(3,1)
    print("numpy: ")
    print(B_hat)
    print('tensorflow: ')
    print(Betas_LR)

numpy: 
[[1.36967319]
 [1.61575796]
 [3.21015672]]
tensorflow: 
[[1.3696747]
 [1.615756 ]
 [3.210157 ]]


## Batch Gradient Descent

1) The `random_uniform()` function creates a node in the graph that will generate a tensor containing random values, given its shape and value range, much like NumPy’s `rand()` function.

2) The `assign()` function creates a node that will assign a new value to a variable. In this case, it implements the Batch Gradient Descent step θ(next step) = θ – η∇θMSE(θ).

3) The main loop executes the training step over and over again (n_epochs times), and every 100 iterations it prints out the current Mean Squared Error (mse). You should see the MSE go down at every iteration.

In [92]:
n_epochs = 1000
learning_rate = 0.01

theta = tf.Variable(tf.random_uniform([3, 1], minval=-5.0, maxval=5.0), name="theta") #Initialize random theta
y_pred = tf.matmul(Xt, theta, name="predictions") #Predictions to calculate MSE
error = y_pred - Yt
mse = tf.reduce_mean(tf.square(error), name="mse") #MSE
gradients = 2/1000 * tf.matmul(Xt_T, error) #Gradient Descent (based on LR Formula)
#gradients = tf.gradients(mse, [theta])[0] --> alternatively use this for automatic partial derivatives (mse/theta)
training_op = tf.assign(theta, theta - learning_rate * gradients) #Assign new value to theta

init = tf.global_variables_initializer() #Initialize all variables when called
saver = tf.train.Saver() #Saver node at the end of the construction phase (after all variable nodes are created)

with tf.Session() as sess:
    sess.run(init) #Initialize variables
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
            save_path = saver.save(sess, "/temp/LR_model.ckpt") #Save model
        sess.run(training_op) #What change the value of theta
    best_theta = theta.eval()

Epoch 0 MSE = 29.205965
Epoch 100 MSE = 26.486671
Epoch 200 MSE = 26.17316
Epoch 300 MSE = 26.001715
Epoch 400 MSE = 25.879038
Epoch 500 MSE = 25.789934
Epoch 600 MSE = 25.725168
Epoch 700 MSE = 25.678078
Epoch 800 MSE = 25.643843
Epoch 900 MSE = 25.618942


In [94]:
with tf.Session() as sess:
    saver.restore(sess, "/temp/LR_model.ckpt") #Restore Model

INFO:tensorflow:Restoring parameters from /temp/LR_model.ckpt


## Artificial Neural Networks (ANNs)

### Training Linear Threshold Units (LTUs)

The Perceptron is fed one training instance at a time, and for each instance it makes its predictions. For every output neuron that produced a wrong prediction, it reinforces the connection weights from the inputs that would have contributed to the correct prediction.

### Backpropagation Algorithm

Gradient Descent using reverse-auto diff.

For each training instance, the algorithm feeds it to the network and computes the output of every neuron in each consecutive layer (this is the forward pass, just like when making predictions). Then it measures the network’s output error (i.e., the difference between the desired output and the actual output of the network), and it computes how much each neuron in the last hidden layer contributed to each output neuron’s error. It then proceeds to measure how much of these error contributions came from each neuron in the previous hidden layer — and so on until the algorithm reaches the input layer. This reverse pass
efficiently measures the error gradient across all the connection weights in the network by propagating the error gradient backward in the network.

In order for this algorithm to work properly, the authors made a key change to the MLP’s architecture: they replaced the step function with the logistic function (stepwise has no gradient).

In [161]:
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import StandardScaler
import numpy as np

#MNIST DATASET
(X_train, Y_train), (X_test, Y_test) = mnist.load_data() #28X28 Images (784 Features)

X_train_clean = []
X_test_clean = []

for image in X_train:
    X_train_clean.append(image.flatten().reshape(784))
X_train_clean = np.array(X_train_clean)

for image in X_test:
    X_test_clean.append(image.flatten().reshape(784))
X_test_clean = np.array(X_test_clean)
    
Y_train_clean = Y_train.reshape(60000,1).astype(int)
Y_test_clean = Y_test.reshape(10000,1).astype(int)

X_tr_std = StandardScaler().fit(X_train_clean).transform(X_train_clean.astype(float))
X_te_std = StandardScaler().fit(X_test_clean).transform(X_test_clean.astype(float))

In [162]:
Y_train_clean.dtype

dtype('int32')

In [163]:
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(X_tr_std)

dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300, 100],
                                         n_classes=10,
                                         feature_columns=feature_columns)

dnn_clf.fit(x=X_tr_std, y=Y_train_clean, batch_size=50, steps=40000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000231A2CAA8D0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\hheim\\AppData\\Local\\Temp\\tmpi0k_9549'}
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.EstimatorSpec. You can use the `estimator_spec` method to create an equivalent one.
INFO:tensorflow:Crea

INFO:tensorflow:loss = 0.009592747, step = 6401 (0.656 sec)
INFO:tensorflow:global_step/sec: 153.082
INFO:tensorflow:loss = 0.004474271, step = 6501 (0.653 sec)
INFO:tensorflow:global_step/sec: 149.875
INFO:tensorflow:loss = 0.013879295, step = 6601 (0.668 sec)
INFO:tensorflow:global_step/sec: 152.896
INFO:tensorflow:loss = 0.0016356269, step = 6701 (0.652 sec)
INFO:tensorflow:global_step/sec: 151.92
INFO:tensorflow:loss = 0.008721442, step = 6801 (0.659 sec)
INFO:tensorflow:global_step/sec: 151.232
INFO:tensorflow:loss = 0.0149595365, step = 6901 (0.661 sec)
INFO:tensorflow:global_step/sec: 148.323
INFO:tensorflow:loss = 0.028747296, step = 7001 (0.675 sec)
INFO:tensorflow:global_step/sec: 132.099
INFO:tensorflow:loss = 0.010895155, step = 7101 (0.755 sec)
INFO:tensorflow:global_step/sec: 150.56
INFO:tensorflow:loss = 0.0010335713, step = 7201 (0.666 sec)
INFO:tensorflow:global_step/sec: 144.471
INFO:tensorflow:loss = 0.0017673171, step = 7301 (0.691 sec)
INFO:tensorflow:global_step/s

INFO:tensorflow:global_step/sec: 154.249
INFO:tensorflow:loss = 0.001320052, step = 14501 (0.651 sec)
INFO:tensorflow:global_step/sec: 148.553
INFO:tensorflow:loss = 5.713666e-05, step = 14601 (0.672 sec)
INFO:tensorflow:global_step/sec: 150.316
INFO:tensorflow:loss = 0.00023745406, step = 14701 (0.664 sec)
INFO:tensorflow:global_step/sec: 154.03
INFO:tensorflow:loss = 0.0008790253, step = 14801 (0.649 sec)
INFO:tensorflow:global_step/sec: 130.979
INFO:tensorflow:loss = 0.00087796716, step = 14901 (0.775 sec)
INFO:tensorflow:global_step/sec: 123.482
INFO:tensorflow:loss = 0.00055776484, step = 15001 (0.805 sec)
INFO:tensorflow:global_step/sec: 161.713
INFO:tensorflow:loss = 0.00081492757, step = 15101 (0.609 sec)
INFO:tensorflow:global_step/sec: 164.113
INFO:tensorflow:loss = 0.0023306936, step = 15201 (0.610 sec)
INFO:tensorflow:global_step/sec: 154.011
INFO:tensorflow:loss = 0.00077050464, step = 15301 (0.651 sec)
INFO:tensorflow:global_step/sec: 152.622
INFO:tensorflow:loss = 0.0010

INFO:tensorflow:loss = 0.000885486, step = 22401 (0.726 sec)
INFO:tensorflow:global_step/sec: 141.614
INFO:tensorflow:loss = 0.00027061233, step = 22501 (0.704 sec)
INFO:tensorflow:global_step/sec: 149.653
INFO:tensorflow:loss = 0.0002888206, step = 22601 (0.667 sec)
INFO:tensorflow:global_step/sec: 151.241
INFO:tensorflow:loss = 0.00029851747, step = 22701 (0.662 sec)
INFO:tensorflow:global_step/sec: 149.811
INFO:tensorflow:loss = 0.00061502727, step = 22801 (0.669 sec)
INFO:tensorflow:global_step/sec: 149.43
INFO:tensorflow:loss = 0.00074131414, step = 22901 (0.667 sec)
INFO:tensorflow:global_step/sec: 151.232
INFO:tensorflow:loss = 5.6412864e-05, step = 23001 (0.665 sec)
INFO:tensorflow:global_step/sec: 150.328
INFO:tensorflow:loss = 0.0005254321, step = 23101 (0.666 sec)
INFO:tensorflow:global_step/sec: 148.54
INFO:tensorflow:loss = 0.00014283547, step = 23201 (0.672 sec)
INFO:tensorflow:global_step/sec: 149.876
INFO:tensorflow:loss = 0.0018695235, step = 23301 (0.666 sec)
INFO:ten

INFO:tensorflow:global_step/sec: 147.663
INFO:tensorflow:loss = 7.9297635e-05, step = 30401 (0.676 sec)
INFO:tensorflow:global_step/sec: 145.321
INFO:tensorflow:loss = 0.00082235935, step = 30501 (0.685 sec)
INFO:tensorflow:global_step/sec: 152.653
INFO:tensorflow:loss = 0.000105793246, step = 30601 (0.657 sec)
INFO:tensorflow:global_step/sec: 150.317
INFO:tensorflow:loss = 0.0010191824, step = 30701 (0.663 sec)
INFO:tensorflow:global_step/sec: 151.014
INFO:tensorflow:loss = 0.00022966268, step = 30801 (0.662 sec)
INFO:tensorflow:global_step/sec: 153.367
INFO:tensorflow:loss = 0.00072786113, step = 30901 (0.652 sec)
INFO:tensorflow:global_step/sec: 150.1
INFO:tensorflow:loss = 0.00037969253, step = 31001 (0.665 sec)
INFO:tensorflow:global_step/sec: 153.081
INFO:tensorflow:loss = 9.659481e-05, step = 31101 (0.653 sec)
INFO:tensorflow:global_step/sec: 149.885
INFO:tensorflow:loss = 0.0011565719, step = 31201 (0.669 sec)
INFO:tensorflow:global_step/sec: 151.453
INFO:tensorflow:loss = 0.00

INFO:tensorflow:loss = 0.00052631187, step = 38301 (0.720 sec)
INFO:tensorflow:global_step/sec: 135.315
INFO:tensorflow:loss = 0.00035034603, step = 38401 (0.739 sec)
INFO:tensorflow:global_step/sec: 136.79
INFO:tensorflow:loss = 8.1963015e-05, step = 38501 (0.728 sec)
INFO:tensorflow:global_step/sec: 137.919
INFO:tensorflow:loss = 0.00034950898, step = 38601 (0.725 sec)
INFO:tensorflow:global_step/sec: 138.866
INFO:tensorflow:loss = 3.162105e-05, step = 38701 (0.725 sec)
INFO:tensorflow:global_step/sec: 138.118
INFO:tensorflow:loss = 0.0006412005, step = 38801 (0.728 sec)
INFO:tensorflow:global_step/sec: 140.43
INFO:tensorflow:loss = 0.0007560246, step = 38901 (0.705 sec)
INFO:tensorflow:global_step/sec: 143.445
INFO:tensorflow:loss = 0.00034432427, step = 39001 (0.699 sec)
INFO:tensorflow:global_step/sec: 137.351
INFO:tensorflow:loss = 0.00018933148, step = 39101 (0.730 sec)
INFO:tensorflow:global_step/sec: 134.408
INFO:tensorflow:loss = 0.00025392688, step = 39201 (0.740 sec)
INFO:t

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x00000231A2DD8828>, 'hidden_units': [300, 100], 'feature_columns': (_RealValuedColumn(column_name='', dimension=784, default_value=None, dtype=tf.float64, normalizer=None),), 'optimizer': None, 'activation_fn': <function relu at 0x0000023187437950>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

In [164]:
from sklearn.metrics import accuracy_score

y_pred = list(dnn_clf.predict(X_te_std))
accuracy_score(Y_test_clean, y_pred)

Instructions for updating:
Please switch to predict_classes, or set `outputs` argument.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\hheim\AppData\Local\Temp\tmpi0k_9549\model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


0.9785